# Beyond The Crosshair

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import json
import ast

alt.data_transformers.enable("default", max_rows=None)
alt.renderers.enable("mimetype")
alt.theme.enable("fivethirtyeight")

ThemeRegistry.enable('fivethirtyeight')

## Datasets & Cleaning

### Counter-Strike Playerbase Data from SteamDB

In [2]:
cs_data = pd.read_csv("../assets/SteamDB Counter-Strike.csv")
cs2_data = pd.read_csv("../assets/SteamDB Counter-Strike 2.csv")
css_data = pd.read_csv("../assets/SteamDB Counter-Strike Source.csv")
cscz_data = pd.read_csv("../assets/SteamDB Counter-Strike Condition Zero.csv")

In [3]:
cs_data_with_dash = cs_data[cs_data["gain"] == "-"]
cs_data_with_dash

,month,peak,gain,%gain,average,average % gain
241,Jan-04,"23,241",-,-,-,-


In [4]:
cs_data[['peak', 'gain', 'average']] = cs_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
cs2_data[['peak', 'gain', 'average']] = cs2_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
css_data[['peak', 'gain', 'average']] = css_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
cscz_data[['peak', 'gain', 'average']] = cscz_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)

fields_to_keep = ['month', 'peak', 'gain', 'average']
cs_data = cs_data[fields_to_keep][1:]
cs2_data = cs2_data[fields_to_keep][1:]
css_data = css_data[fields_to_keep][1:]
cscz_data = cscz_data[fields_to_keep][1:]

split_date = pd.Timestamp("2023-09-01")



cs_data["month"] = pd.to_datetime(cs_data["month"], format="%b-%y")
cs_data = cs_data.sort_values("month")
css_data["month"] = pd.to_datetime(css_data["month"], format="%b-%y")
css_data = css_data.sort_values("month")
cscz_data["month"] = pd.to_datetime(cscz_data["month"], format="%b-%y")
cscz_data = cscz_data.sort_values("month")
cs2_data["month"] = pd.to_datetime(cs2_data["month"], format="%b-%y")
cs2_data = cs2_data.sort_values("month")

csgo_data = cs2_data[cs2_data["month"] < split_date]
cs2_data = cs2_data[cs2_data["month"] >= split_date]

csgo_data["month"] = pd.to_datetime(csgo_data["month"], format="%b-%y")
csgo_data = csgo_data.sort_values("month")

In [5]:
# fields_game_versions = ['Counter-Strike', 'Counter-Strike 2', 'Counter-Strike Source', 'Counter-Strike Condition Zero']

cs_merged = pd.concat([cs_data, cs2_data, csgo_data, css_data, cscz_data], ignore_index=True)
cs_merged = cs_merged[fields_to_keep]
cs_merged.head()

,month,peak,gain,average
0,2004-01-01,23241.0,0.0,0.0
1,2004-02-01,39709.0,16468.0,0.0
2,2004-04-01,74926.0,35217.0,0.0
3,2004-06-01,66876.0,8050.0,0.0
4,2004-07-01,69685.0,2809.0,0.0


In [6]:
SteamDB_merged_grouped = cs_merged.groupby(cs_merged.columns[0]).sum()

SteamDB_merged_grouped = SteamDB_merged_grouped.reset_index()
SteamDB_merged_grouped = SteamDB_merged_grouped.sort_values("month")
SteamDB_merged_grouped

,month,peak,gain,average
0,2004-01-01,23241.0,0.0,0.0
1,2004-02-01,39709.0,16468.0,0.0
2,2004-04-01,84182.0,35217.0,0.0
3,2004-06-01,80908.0,12826.0,0.0
4,2004-07-01,84804.0,3896.0,0.0
...,...,...,...,...
236,2025-04-01,1898525.0,40987.0,1070094.0
237,2025-05-01,1752347.0,146178.0,1060039.0
238,2025-06-01,1766572.0,14429.0,1029242.0
239,2025-07-01,1506027.0,338169.0,954251.0


In [7]:
# cs_data.to_csv("../assets/SteamDB_cs_cleaned.csv",encoding="utf-8")
# css_data.to_csv("../assets/SteamDB_css_cleaned.csv",encoding="utf-8")
# cscz_data.to_csv("../assets/SteamDB_cscz_cleaned.csv",encoding="utf-8")
# csgo_data.to_csv("../assets/SteamDB_csgo_cleaned.csv",encoding="utf-8")
# cs2_data.to_csv("../assets/SteamDB_cs2_cleaned.csv",encoding="utf-8")
# SteamDB_merged_grouped.to_csv("../assets/SteamDB_merged_cleaned.csv",encoding="utf-8")

### Rival Game Data from IGDB

In [8]:
"""
Used to split igdb data into 3 separate parts because of github upload limit, commented out after use
"""

# game_data = pd.read_csv("../assets/igdb_shooters.csv")

# splitting all genres csv into separate csv files

# Replace this with the path to your big CSV
# input_file = "../assets/igdb_games.csv"

# # Read the CSV into a DataFrame
# df = pd.read_csv(input_file)

# # Calculate the size of each split
# n = len(df)
# split_size = n // 3

# # Split into 3 parts
# df1 = df.iloc[:split_size]
# df2 = df.iloc[split_size:2*split_size]
# df3 = df.iloc[2*split_size:]

# Save to separate CSV files
# df1.to_csv("../assets/igdb_part1.csv", index=False)
# df2.to_csv("../assets/igdb_part2.csv", index=False)
# df3.to_csv("../assets/igdb_part3.csv", index=False)

# print("Splitting complete! Files saved as part1.csv, part2.csv, part3.csv")

'\nUsed to split igdb data into 3 separate parts because of github upload limit, commented out after use\n'

In [9]:
igdb_data_1 = pd.read_csv("../assets/igdb_part1.csv")
igdb_data_2 = pd.read_csv("../assets/igdb_part2.csv")
igdb_data_3 = pd.read_csv("../assets/igdb_part3.csv")
igdb_data_merged = pd.concat([igdb_data_1, igdb_data_2, igdb_data_3], ignore_index=True)

/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_33500/2886035041.py:1: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  igdb_data_1 = pd.read_csv("../assets/igdb_part1.csv")
/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_33500/2886035041.py:2: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  igdb_data_2 = pd.read_csv("../assets/igdb_part2.csv")
/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_33500/2886035041.py:3: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  igdb_data_3 = pd.read_csv("../assets/igdb_part3.csv")


In [10]:
IGDB_fields = ['id', 'name', 'first_release_date', 'genres', 'rating', 'rating_count', 'total_rating', 'total_rating_count', 'aggregated_rating',
       'aggregated_rating_count']

igdb_data_merged_filtered = igdb_data_merged[IGDB_fields]
igdb_data_merged_filtered["first_release_date"] = pd.to_datetime(igdb_data_merged_filtered["first_release_date"], unit="s") # Formatting date

/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_33500/1111918781.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  igdb_data_merged_filtered["first_release_date"] = pd.to_datetime(igdb_data_merged_filtered["first_release_date"], unit="s") # Formatting date


In [11]:
vanilla_wow = igdb_data_merged_filtered[igdb_data_merged_filtered['id']==123] # Vanilla WoW
cata_wow = igdb_data_merged_filtered[igdb_data_merged_filtered['id']==229] # Cataclysm
cata_wow

,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
327330,229,World of Warcraft: Cataclysm,2010-12-07,[12],76.861351,198.0,83.097342,201.0,89.333333,3.0


In [12]:
with open("../data/genres.json", "r", encoding="utf-8") as f:
    genres_list = json.load(f)
    genres_list = {int(k): v for k, v in genres_list.items()}

genres_list

{31: 'Adventure',
 33: 'Arcade',
 35: 'Card & Board Game',
 4: 'Fighting',
 25: "Hack and slash/Beat 'em up",
 32: 'Indie',
 36: 'MOBA',
 7: 'Music',
 30: 'Pinball',
 8: 'Platform',
 2: 'Point-and-click',
 9: 'Puzzle',
 26: 'Quiz/Trivia',
 10: 'Racing',
 11: 'Real Time Strategy (RTS)',
 12: 'Role-playing (RPG)',
 5: 'Shooter',
 13: 'Simulator',
 14: 'Sport',
 15: 'Strategy',
 24: 'Tactical',
 16: 'Turn-based strategy (TBS)',
 34: 'Visual Novel'}

In [13]:
igdb_data_merged_filtered['genres'] = igdb_data_merged_filtered['genres'].map(lambda x: [] if pd.isna(x) else[genres_list[g] for g in x] if isinstance(x, list) else [genres_list[g] for g in ast.literal_eval(x)])
igdb_data_merged_filtered.head()

/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_33500/129189592.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  igdb_data_merged_filtered['genres'] = igdb_data_merged_filtered['genres'].map(lambda x: [] if pd.isna(x) else[genres_list[g] for g in x] if isinstance(x, list) else [genres_list[g] for g in ast.literal_eval(x)])


,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
0,165499,^_^,2012-01-14,[Adventure],NaN,NaN,NaN,NaN,NaN,NaN
1,141830,||[}}}°.•°.°•..°•°[|||{{{,2020-02-02,[Indie],NaN,NaN,NaN,NaN,NaN,NaN
2,191680,_____,2014-12-11,"[Shooter, Indie]",NaN,NaN,NaN,NaN,NaN,NaN
3,176875,__________,2018-07-05,"[Adventure, Indie]",NaN,NaN,NaN,NaN,NaN,NaN
4,92204,_-_,2018-03-14,"[Puzzle, Arcade]",90.0,0.0,90.0,0.0,NaN,NaN


In [14]:
# Filtering entries without rating
print(f"With and without rating = {len(igdb_data_merged_filtered)}")
igdb_data_merged_filtered_converted = igdb_data_merged_filtered.dropna(subset=['rating'])
print(f"With rating = {len(igdb_data_merged_filtered_converted)}")

# Filtering entries without first_release_date
print(f"With and without first_release_date = {len(igdb_data_merged_filtered_converted)}")
igdb_data_merged_filtered_converted = igdb_data_merged_filtered_converted[~igdb_data_merged_filtered_converted["first_release_date"].isna()]
print(f"With first_release_date = {len(igdb_data_merged_filtered_converted)}")

# Filtering entries before 2004
igdb_data_merged_filtered_converted = igdb_data_merged_filtered_converted[pd.to_datetime(igdb_data_merged_filtered_converted["first_release_date"]) > pd.to_datetime("2004-01-01")]
igdb_data_merged_filtered_converted.head()

With and without rating = 335086
With rating = 32671
With and without first_release_date = 32671
With first_release_date = 31702


,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
4,92204,_-_,2018-03-14,"[Puzzle, Arcade]",90.000000,0.0,90.000000,0.0,NaN,NaN
23,1649,007 Legends,2012-10-16,[Shooter],54.172800,23.0,49.943543,30.0,45.714286,7.0
48,113188,03.04,2019-01-01,"[Adventure, Indie]",60.000000,2.0,60.000000,2.0,NaN,NaN
69,55486,0°N 0°W,2018-03-02,"[Adventure, Indie]",70.000000,4.0,60.000000,5.0,50.000000,1.0
72,9583,0rbitalis,2014-04-16,"[Puzzle, Simulator, Indie]",79.935551,5.0,77.467775,8.0,75.000000,3.0


In [58]:
igdb_data_merged_filtered_converted.sort_values(by="total_rating_count", ascending=False).head(50)

,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
117313,1020,Grand Theft Auto V,2013-09-17,"[Shooter, Racing, Adventure]",89.554068,5200.0,88.846000,5227.0,88.137931,27.0
295137,1942,The Witcher 3: Wild Hunt,2015-05-19,"[Role-playing (RPG), Adventure]",93.969921,4754.0,92.850345,4780.0,91.730769,26.0
215199,72,Portal 2,2011-04-18,"[Platform, Puzzle, Adventure]",91.539118,3997.0,91.991781,4006.0,92.444444,9.0
283973,472,The Elder Scrolls V: Skyrim,2011-11-10,"[Role-playing (RPG), Adventure]",87.491280,3872.0,83.703973,3882.0,79.916667,10.0
215195,71,Portal,2007-10-10,"[Shooter, Platform, Puzzle]",86.445621,3609.0,83.222811,3610.0,80.000000,1.0
117301,732,Grand Theft Auto: San Andreas,2004-10-26,"[Shooter, Racing, Adventure]",90.231380,3508.0,91.687118,3514.0,93.142857,6.0
287598,1009,The Last of Us,2013-06-14,"[Shooter, Adventure]",93.097059,3063.0,92.317760,3076.0,91.538462,13.0
121518,233,Half-Life 2,2004-11-16,[Shooter],90.410008,3050.0,84.767504,3057.0,79.125000,7.0
226686,25076,Red Dead Redemption 2,2018-10-26,"[Shooter, Role-playing (RPG), Adventure]",93.276503,3016.0,93.667663,3033.0,94.058824,17.0
115004,19560,God of War,2018-04-20,"[Role-playing (RPG), Hack and slash/Beat 'em u...",92.362610,2996.0,94.296689,3022.0,96.230769,26.0


In [15]:
games_with_counter = igdb_data_merged_filtered_converted[igdb_data_merged_filtered_converted["name"].str.contains("counter", case=False, na=False)]
games_with_counter

,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
58976,33278,CounterAttack,2016-04-21,"[Shooter, Adventure, Indie, Arcade]",80.000000,1.0,80.000000,1.0,NaN,NaN
58993,30643,Counter Fight,2016-11-14,"[Simulator, Indie]",80.000000,0.0,80.000000,0.0,NaN,NaN
58997,40553,Counter Fight: Samurai Edition,2017-06-07,"[Simulator, Indie]",70.000000,0.0,70.000000,0.0,NaN,NaN
59023,35703,Counter Spell,2015-10-05,"[Role-playing (RPG), Indie]",80.000000,0.0,80.000000,0.0,NaN,NaN
59025,7612,CounterSpy,2014-08-19,"[Shooter, Tactical, Indie]",75.046860,36.0,69.210930,44.0,63.375000,8.0
59028,242408,Counter-Strike 2,2023-09-27,"[Shooter, Tactical]",67.185101,249.0,67.185101,249.0,NaN,NaN
59031,297,Counter-Strike: Condition Zero,2004-03-23,[Shooter],74.174541,217.0,70.253937,220.0,66.333333,3.0
59032,93807,Counter-Strike: Condition Zero Deleted Scenes,2004-03-23,[Shooter],64.784238,32.0,64.784238,32.0,NaN,NaN
59033,1372,Counter-Strike: Global Offensive,2012-08-21,"[Shooter, Tactical]",81.813980,1245.0,81.406990,1250.0,81.000000,5.0
59037,16957,Counter-Strike Nexon,2014-10-07,"[Shooter, Indie]",20.907759,22.0,20.907759,22.0,NaN,NaN


In [16]:
# igdb_data_merged_filtered_converted.to_csv("../assets/IGDB_merged_cleaned.csv", index=False)

### Twitch Viewership Data from TwitchTracker

In [17]:
cs_twitch_data = pd.read_csv("../assets/TwitchTracker Counter-Strike.csv")
cs2_twitch_data = pd.read_csv("../assets/TwitchTracker Counter-Strike 2.csv")
cscz_twitch_data = pd.read_csv("../assets/TwitchTracker Counter-Strike Condition Zero.csv")
css_twitch_data = pd.read_csv("../assets/TwitchTracker Counter-Strike Source.csv")

twitch_fields = ['Month', 'Avg Viewers', 'Gain', 'Peak Viewers', 'Avg Streams', 'Gain.1', 'Peak Streams', 'Hours Watched']

In [18]:
twitch_data_merged = pd.concat([cs_twitch_data, cs2_twitch_data, cscz_twitch_data, css_twitch_data], ignore_index=True)
twitch_data_merged = twitch_data_merged[twitch_fields]
twitch_data_merged

,Month,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched
0,Nov-16,18.0,-,164,2.0,-,7.0,3.2K
1,Dec-16,20.0,2,509,2.0,-,10.0,8.0K
2,Jan-17,19.0,-1,543,4.0,2,13.0,8.8K
3,Feb-17,30.0,11,"5,782",2.0,-2,11.0,16.0K
4,Mar-17,18.0,-12,815,2.0,-,10.0,6.9K
...,...,...,...,...,...,...,...,...
316,Jan-17,18,-1,215,1,-,3,357
317,Dec-16,19,4,216,1,-,3,946
318,Nov-16,15,-,147,1,-,3,336
319,Aug-25,14,-,"1,213",2,-,8,4.4K


In [19]:
# Transforming Hours Watched format
def convert_k(val):
    if isinstance(val, str):
        if val.endswith("K"):
            return float(val[:-1].replace(".", ""))* 1000
        elif val.endswith("M"):
            return float(val[:-1].replace(".", "")) * 1000000
    return float(val)

twitch_data_merged["Hours Watched"] = twitch_data_merged["Hours Watched"].map(convert_k)
twitch_data_merged

,Month,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched
0,Nov-16,18.0,-,164,2.0,-,7.0,32000.0
1,Dec-16,20.0,2,509,2.0,-,10.0,80000.0
2,Jan-17,19.0,-1,543,4.0,2,13.0,88000.0
3,Feb-17,30.0,11,"5,782",2.0,-2,11.0,160000.0
4,Mar-17,18.0,-12,815,2.0,-,10.0,69000.0
...,...,...,...,...,...,...,...,...
316,Jan-17,18,-1,215,1,-,3,357.0
317,Dec-16,19,4,216,1,-,3,946.0
318,Nov-16,15,-,147,1,-,3,336.0
319,Aug-25,14,-,"1,213",2,-,8,44000.0


In [20]:
numeric_fields = [x for x in twitch_fields if x != "Month"]
twitch_data_merged[numeric_fields] = twitch_data_merged[numeric_fields].replace({",": "", "-": "0"}, regex=True).astype(float)

twitch_data_merged["Month"] = pd.to_datetime(twitch_data_merged["Month"], format="%b-%y")
twitch_data_merged = twitch_data_merged.sort_values("Month")
twitch_data_merged = twitch_data_merged.rename(columns={"Month": "month"})
twitch_data_merged.head()

,month,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched
0,2016-11-01,18.0,0.0,164.0,2.0,0.0,7.0,32000.0
318,2016-11-01,15.0,0.0,147.0,1.0,0.0,3.0,336.0
212,2016-11-01,41089.0,0.0,236838.0,800.0,0.0,1723.0,305000000.0
211,2016-12-01,97356.0,56267.0,1153507.0,937.0,137.0,2172.0,721000000.0
1,2016-12-01,20.0,2.0,509.0,2.0,0.0,10.0,80000.0


In [21]:
twitch_data_merged[twitch_data_merged['month'].dt.year == 2023]

,month,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched
244,2023-01-01,3.0,1.0,45.0,1.0,0.0,4.0,319.0
74,2023-01-01,131.0,15.0,2109.0,13.0,1.0,36.0,875000.0
138,2023-01-01,82042.0,29722.0,576269.0,1176.0,80.0,2245.0,545000000.0
75,2023-02-01,139.0,8.0,7415.0,13.0,0.0,38.0,103000.0
137,2023-02-01,88932.0,6890.0,883067.0,1340.0,164.0,2957.0,655000000.0
243,2023-02-01,2.0,1.0,30.0,1.0,0.0,3.0,412.0
136,2023-03-01,112611.0,23679.0,579500.0,1580.0,240.0,3049.0,808000000.0
242,2023-03-01,2.0,0.0,66.0,1.0,0.0,4.0,536.0
76,2023-03-01,152.0,13.0,9555.0,13.0,0.0,35.0,109000.0
77,2023-04-01,146.0,6.0,5348.0,13.0,0.0,33.0,109000.0


In [22]:
twitch_data_merged_grouped = twitch_data_merged.groupby(twitch_data_merged.columns[0]).sum()
twitch_data_merged_grouped = twitch_data_merged_grouped.reset_index()
# twitch_data_merged_grouped.to_csv("../assets/TwitchTracker_merged_cleaned.csv", encoding="utf-8")
twitch_data_merged_grouped

,month,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched
0,2016-11-01,41122.0,0.0,237149.0,803.0,0.0,1733.0,305032336.0
1,2016-12-01,97395.0,56273.0,1154232.0,940.0,137.0,2185.0,721080946.0
2,2017-01-01,60177.0,37218.0,461337.0,1014.0,74.0,2097.0,404088357.0
3,2017-02-01,49606.0,10593.0,512456.0,946.0,68.0,2100.0,368160397.0
4,2017-03-01,50868.0,1286.0,418276.0,879.0,69.0,1830.0,364069433.0
...,...,...,...,...,...,...,...,...
101,2025-04-01,99822.0,12660.0,722031.0,1601.0,80.0,3272.0,741902172.0
102,2025-05-01,117782.0,18074.0,1146827.0,1481.0,120.0,3053.0,848466647.0
103,2025-06-01,86343.0,31463.0,556026.0,1519.0,38.0,2998.0,642565317.0
104,2025-07-01,91794.0,5449.0,734295.0,1558.0,37.0,3044.0,682532909.0


### Youtube Video Data from Youtube Data API

In [23]:
youtube_data = pd.read_csv("../assets/yt_counter_strike_monthly_top50.csv")

In [24]:
youtube_data = youtube_data.drop(columns=["month"], errors="ignore")
youtube_data = youtube_data.rename(columns={"publishedAt": "month"})

youtube_fields = ['month', 'title', 'description', 'viewCount', 'likeCount', 'commentCount', 'favoriteCount', 'categoryId']
youtube_data = youtube_data[youtube_fields]
youtube_data["month"] = pd.to_datetime(youtube_data["month"], utc=True).dt.to_period("M").dt.to_timestamp()

youtube_data = youtube_data.sort_values(by="viewCount", ascending=False)

youtube_data['viewCount'] = youtube_data['viewCount'].astype("Int64")
youtube_data = youtube_data[1:]
youtube_data.head()

/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_33500/2895940128.py:6: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  youtube_data["month"] = pd.to_datetime(youtube_data["month"], utc=True).dt.to_period("M").dt.to_timestamp()


,month,title,description,viewCount,likeCount,commentCount,favoriteCount,categoryId
2363,2009-10-01,Counter-Strike - DE dust2 HD,"Based on the game, Counter-Strike: Source, thi...",123912746,1514852.0,99377.0,0,1
8312,2019-09-01,Counter-Strike: Zombie Escape Mod - ze_THELOST...,Counter-Strike 1.6: ze thelostworld bf WIN (4K...,43609790,203296.0,1587.0,0,20
6462,2016-08-01,Counter-Strike: Zombie Escape Mod - ze_Jurassi...,Counter-Strike 1.6: ze_Jurassicpark_v2 WIN (10...,40556603,138034.0,1742.0,0,20
7562,2018-06-01,Ответный удар - Мультики про танки,Мультик про танки по мотивам игры World of Tan...,37482544,122935.0,1941.0,0,20
8412,2019-11-01,Counter Attack Gun Strike Special Ops Shooting...,Counter Attack Gun Strike Special Ops Shooting...,35893795,104716.0,845.0,0,20


In [25]:
# youtube_data.to_csv("../assets/Youtube_merged_cleaned.csv", encoding="utf-8")

In [26]:
youtube_data_sorted_by_viewCount = youtube_data.sort_values("viewCount")
youtube_data_sorted_by_viewCount

,month,title,description,viewCount,likeCount,commentCount,favoriteCount,categoryId
61,2005-11-01,Nikeny64's Montage,nikeny64's montage,123,1.0,2.0,0,24
60,2005-11-01,Sparty16's Montage,It is a snipe fest.,327,8.0,1.0,0,24
13,2005-08-01,rfintro,Intro,436,4.0,5.0,0,24
14,2005-08-01,Dierlect,all about the map dierlect,436,2.0,1.0,0,24
12,2005-08-01,MULLETAGE,this is Chris Mo Fo Coles w/ his new updated m...,505,5.0,1.0,0,1
...,...,...,...,...,...,...,...,...
7562,2018-06-01,Ответный удар - Мультики про танки,Мультик про танки по мотивам игры World of Tan...,37482544,122935.0,1941.0,0,20
6462,2016-08-01,Counter-Strike: Zombie Escape Mod - ze_Jurassi...,Counter-Strike 1.6: ze_Jurassicpark_v2 WIN (10...,40556603,138034.0,1742.0,0,20
8312,2019-09-01,Counter-Strike: Zombie Escape Mod - ze_THELOST...,Counter-Strike 1.6: ze thelostworld bf WIN (4K...,43609790,203296.0,1587.0,0,20
2363,2009-10-01,Counter-Strike - DE dust2 HD,"Based on the game, Counter-Strike: Source, thi...",123912746,1514852.0,99377.0,0,1


## Visualizations

In [27]:
SteamDB_merged_grouped.head()

,month,peak,gain,average
0,2004-01-01,23241.0,0.0,0.0
1,2004-02-01,39709.0,16468.0,0.0
2,2004-04-01,84182.0,35217.0,0.0
3,2004-06-01,80908.0,12826.0,0.0
4,2004-07-01,84804.0,3896.0,0.0


In [28]:
igdb_data_merged_filtered_converted.head()

,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
4,92204,_-_,2018-03-14,"[Puzzle, Arcade]",90.000000,0.0,90.000000,0.0,NaN,NaN
23,1649,007 Legends,2012-10-16,[Shooter],54.172800,23.0,49.943543,30.0,45.714286,7.0
48,113188,03.04,2019-01-01,"[Adventure, Indie]",60.000000,2.0,60.000000,2.0,NaN,NaN
69,55486,0°N 0°W,2018-03-02,"[Adventure, Indie]",70.000000,4.0,60.000000,5.0,50.000000,1.0
72,9583,0rbitalis,2014-04-16,"[Puzzle, Simulator, Indie]",79.935551,5.0,77.467775,8.0,75.000000,3.0


In [29]:
twitch_data_merged_grouped.head()

,month,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched
0,2016-11-01,41122.0,0.0,237149.0,803.0,0.0,1733.0,305032336.0
1,2016-12-01,97395.0,56273.0,1154232.0,940.0,137.0,2185.0,721080946.0
2,2017-01-01,60177.0,37218.0,461337.0,1014.0,74.0,2097.0,404088357.0
3,2017-02-01,49606.0,10593.0,512456.0,946.0,68.0,2100.0,368160397.0
4,2017-03-01,50868.0,1286.0,418276.0,879.0,69.0,1830.0,364069433.0


In [30]:
youtube_data.head()

,month,title,description,viewCount,likeCount,commentCount,favoriteCount,categoryId
2363,2009-10-01,Counter-Strike - DE dust2 HD,"Based on the game, Counter-Strike: Source, thi...",123912746,1514852.0,99377.0,0,1
8312,2019-09-01,Counter-Strike: Zombie Escape Mod - ze_THELOST...,Counter-Strike 1.6: ze thelostworld bf WIN (4K...,43609790,203296.0,1587.0,0,20
6462,2016-08-01,Counter-Strike: Zombie Escape Mod - ze_Jurassi...,Counter-Strike 1.6: ze_Jurassicpark_v2 WIN (10...,40556603,138034.0,1742.0,0,20
7562,2018-06-01,Ответный удар - Мультики про танки,Мультик про танки по мотивам игры World of Tan...,37482544,122935.0,1941.0,0,20
8412,2019-11-01,Counter Attack Gun Strike Special Ops Shooting...,Counter Attack Gun Strike Special Ops Shooting...,35893795,104716.0,845.0,0,20


In [31]:
youtube_data_views_by_month = youtube_data.groupby("month", as_index=False)["viewCount"].sum()
youtube_data_views_by_month

,month,viewCount
0,2005-08-01,89144
1,2005-09-01,19415
2,2005-10-01,263357
3,2005-11-01,1810148
4,2005-12-01,10797827
...,...,...
190,2021-06-01,7542215
191,2021-07-01,3258535
192,2021-08-01,4094505
193,2021-09-01,8603503


In [32]:
# MERGING YOTUBE AND STEAMDB
merged_yt_and_steamdb = pd.merge(SteamDB_merged_grouped[['month','peak']], youtube_data_views_by_month[['month','viewCount']], on="month", how="inner")
merged_yt_and_steamdb = merged_yt_and_steamdb.sort_values(by="viewCount", ascending=False)
merged_yt_and_steamdb

,month,peak,viewCount
33,2009-10-01,158107.0,200082263
115,2016-08-01,631402.0,57190017
137,2018-06-01,449409.0,55513661
152,2019-09-01,745425.0,52619174
135,2018-04-01,556019.0,52530015
...,...,...,...
39,2010-04-01,167923.0,1680624
81,2013-10-01,166374.0,1423714
140,2018-09-01,612880.0,822816
0,2005-08-01,205663.0,89144


In [33]:
# MERGING TWITCH AND STEAMDB
SteamDB_merged_grouped_cutoff = SteamDB_merged_grouped[SteamDB_merged_grouped['month'] >= pd.to_datetime("2016-11-01")]
merged_twitch_and_steamdb = pd.merge(SteamDB_merged_grouped_cutoff[['month','peak']], twitch_data_merged_grouped[['month','Peak Viewers']], on="month", how="inner")
merged_twitch_and_steamdb = merged_twitch_and_steamdb.sort_values(by="Peak Viewers", ascending=False)
merged_twitch_and_steamdb

,month,peak,Peak Viewers
59,2021-10-01,888099.0,1950442.0
65,2022-04-01,1042536.0,1634605.0
87,2024-02-01,1431938.0,1553750.0
13,2017-12-01,638526.0,1320676.0
71,2022-10-01,1110627.0,1273072.0
...,...,...,...
43,2020-06-01,986980.0,211031.0
8,2017-07-01,670361.0,210176.0
25,2018-12-01,781335.0,201034.0
5,2017-04-01,728537.0,194939.0


In [34]:
# cs_data
# css_data
# cscz_data
# csgo_data
# cs2_data
# SteamDB_merged_grouped

In [35]:
SteamDB_merged_grouped

,month,peak,gain,average
0,2004-01-01,23241.0,0.0,0.0
1,2004-02-01,39709.0,16468.0,0.0
2,2004-04-01,84182.0,35217.0,0.0
3,2004-06-01,80908.0,12826.0,0.0
4,2004-07-01,84804.0,3896.0,0.0
...,...,...,...,...
236,2025-04-01,1898525.0,40987.0,1070094.0
237,2025-05-01,1752347.0,146178.0,1060039.0
238,2025-06-01,1766572.0,14429.0,1029242.0
239,2025-07-01,1506027.0,338169.0,954251.0


In [36]:
series = {
    # "All Counter-Strike (aggregate)": SteamDB_merged_grouped,
    "Counter-Strike (1.6)": cs_data,
    "Counter-Strike: Source": css_data,
    "Counter-Strike: Condition Zero": cscz_data,
    "Counter-Strike: Global Offensive": csgo_data,
    "Counter-Strike 2": cs2_data,
}

colors = {
    # "All Counter-Strike (aggregate)":  "#0057B8",  # deep royal blue (highlighted)
    "Counter-Strike (1.6)":            "#009E73",  # green
    "Counter-Strike: Condition Zero":  "#7F7F7F",  # medium gray (subtle)
    "Counter-Strike: Source":          "#56B4E9",  # sky blue
    "Counter-Strike: Global Offensive":"#D55E00",  # vermillion (strong contrast vs blue)
    "Counter-Strike 2":                "#CC79A7",  # magenta
}

order = [
#     "All Counter-Strike (aggregate)",
    "Counter-Strike (1.6)",
    "Counter-Strike: Condition Zero",
    "Counter-Strike: Source",
    "Counter-Strike: Global Offensive",
    "Counter-Strike 2",
]

# Combine all series into one tidy DataFrame
frames = []
for name, df in series.items():
    df["game"] = name
    frames.append(df[["month", "peak", "game"]])

combined = pd.concat(frames, ignore_index=True)

# Build Altair chart
chart = (
    alt.Chart(combined)
    .mark_line()
    .encode(
        x=alt.X("month:T", title="Year"),
        y=alt.Y("peak:Q", title="Peak players"),
        color=alt.Color("game:N", title="Game/Version", scale=alt.Scale(domain=order, range=[colors[k] for k in order])),
        tooltip=["game", "month", "peak"],
    )
    .properties(width=950, height=450, title="Counter-Strike Playerbase Over Time")
)

chart

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [43]:
merged_yt_twitch = pd.concat([twitch_data_merged_grouped.set_index("month"), youtube_data_views_by_month.set_index("month")], axis=1).reset_index()
merged_yt_twitch.head()

,month,Avg Viewers,Gain,Peak Viewers,Avg Streams,Gain.1,Peak Streams,Hours Watched,viewCount
0,2005-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89144
1,2005-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19415
2,2005-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,263357
3,2005-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1810148
4,2005-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10797827


In [51]:
# First line: views (left y-axis)
twitch_line = alt.Chart(merged_yt_twitch).mark_line(color="purple").encode(
    x=alt.X("month:T", title="Month"),
    y=alt.Y("Peak Viewers:Q", title="Twitch", axis=alt.Axis(titleColor="purple"))
)

# Second line: likes (right y-axis)
youtube_line = alt.Chart(merged_yt_twitch).mark_line(color="red").encode(
    x="month:T",
    y=alt.Y("viewCount:Q", title="Youtube", axis=alt.Axis(titleColor="red")),
)

# Layer the two charts
chart = alt.layer(youtube_line, twitch_line).resolve_scale(
    y="independent"  # independent y-axes so they can differ
).properties(width=950, height=450, title="Counter-Strike Viewership Over Time")

chart

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [39]:
# Ensure proper dtypes
df1 = merged_yt_and_steamdb.copy()
df1["month"] = pd.to_datetime(df1["month"])
df1["year"] = df1["month"].dt.year

# Base scatter
scatter = (
    alt.Chart(df1)
    .mark_circle(size=60, opacity=0.6)
    .encode(
        x=alt.X("peak:Q", title="Playerbase (monthly peak, SteamDB)"),
        y=alt.Y("viewCount:Q", title="YouTube Viewership (monthly total)"),
        color=alt.Color("year:N", title="Year", legend=alt.Legend(columns=2)),
        tooltip=[
            alt.Tooltip("month:T", title="Month"),
            alt.Tooltip("peak:Q", title="Peak players", format=","),
            alt.Tooltip("viewCount:Q", title="Views", format=","),
        ]
    )
)

# Regression trendline (linear)
trend = (
    scatter
    .transform_regression("peak", "viewCount")
    .mark_line(size=3)
    .encode(color=alt.value("black"))
)

(scatter + trend).properties(
    title="Counter-Strike: Playerbase vs. YouTube Viewership (by month)"
)


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [40]:
merged_twitch_and_steamdb.head()

,month,peak,Peak Viewers
59,2021-10-01,888099.0,1950442.0
65,2022-04-01,1042536.0,1634605.0
87,2024-02-01,1431938.0,1553750.0
13,2017-12-01,638526.0,1320676.0
71,2022-10-01,1110627.0,1273072.0


In [42]:
# Ensure proper dtypes
df2 = merged_twitch_and_steamdb.copy()
df2["month"] = pd.to_datetime(df2["month"])
df2["year"] = df2["month"].dt.year

# Base scatter
scatter = (
    alt.Chart(df2)
    .mark_circle(size=60, opacity=0.6)
    .encode(
        x=alt.X("peak:Q", title="Playerbase (monthly peak, SteamDB)"),
        y=alt.Y("Peak Viewers:Q", title="Twitch Peak Viewership (monthly total)"),
        color=alt.Color("year:N", title="Year", legend=alt.Legend(columns=2)),
        tooltip=[
            alt.Tooltip("month:T", title="Month"),
            alt.Tooltip("peak:Q", title="Peak players", format=","),
            alt.Tooltip("Peak Viewers:Q", title="Peak Viewers", format=","),
        ]
    )
)

# Regression trendline (linear)
trend = (
    scatter
    .transform_regression("peak", "Peak Viewers")
    .mark_line(size=3)
    .encode(color=alt.value("black"))
)

(scatter + trend).properties(
    title="Counter-Strike: Playerbase vs. Twitch Peak Viewership (by month)"
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting
